In [ ]:
from unicodedata import category, normalize

letter = {'L'}
space = {'Z'}
letter_space = {'L', 'Z'}
dia = {'M'}
punc = {'P'}
letter_dia = {'L', 'M'}
udnorm = 'NFC'

test1a = '-:Κεκρ;?ότη-ται᾿,᾿'
test1b = 'Κεκρ;?ότηται᾿,᾿'
test1c = '-:Κε.:κρ;?ότη,.τα...ι᾿,᾿'
test2 = '''   Κεκ.,ρότη-ται;?    ?κρη/πὶς ..ἀληθείας, ;ὦ παῖδες ὑμεῖς, ἡμῖν αὐτοῖς, 
ἁγίου νεὼ μεγάλου θεοῦ θεμέλιος γνώσεως ἀρραγής, προτροπὴ καλή, 
δι᾿ ὑπακοῆς εὐλόγου ζωῆς ἀιδίουὄρεξις, νοερῷ καταβληθεῖσα χωρίῳ.'''

In [ ]:
def rsplitPunc(word, norm=udnorm, clean=False):
    '''This function splits off punctuation 
    from words on the RIGHT side of the word.
    
    returns (word, punc)
    '''
    w = normalize(norm, word)
    afterWord = len(w)
    for i in range(len(w) - 1, -1, -1):
        if category(w[i])[0] not in letter_dia:
            afterWord = i
        else:
            break
    if clean:
        return (''.join(c for c in w[0:afterWord] \
                          if category(c)[0] in letter_dia), w[afterWord:])
    else:
        return (w[0:afterWord], w[afterWord:])

In [ ]:
print(rsplitPunc(test1a, clean=True))

In [ ]:
def lsplitPunc(word, norm=udnorm, clean=False):
    '''This function splits off punctuation 
    from words on the LEFT side of the word.
    
    returns (punc, word)
    '''
    w = normalize(norm, word)
    beforeWord = -1
    for i in range(len(w)):
        if category(w[i])[0] not in letter_dia:
            beforeWord = i
        else:
            beforeWord +=1
            break
    if clean:
        return (w[0:beforeWord], ''.join(c for c in w[beforeWord:] \
                                           if category(c)[0] in letter_dia))
    else:
        return (w[0:beforeWord], w[beforeWord:])


In [ ]:
print(lsplitPunc(test1a, clean=True))


In [ ]:
def splitPunc(words, norm=udnorm, clean=False,
              splitters=None, non_splitters=None):
    '''This function splits off punctuation 
    from words on both sides of the word. 
    It returns a tuple with tuples, containing
    the punctuation before, the word itself, 
    and punctuation after. It can be used for
    multiple words
    
    clean=False:
        if punctuation is within the word, the word
        will be split into two, except for characters
        defined in the non-splitters list.
    clean=True:
        punctuation within a word will be deleted, 
        except for characters defined in the splitters 
        list. In that case, the string will be split.
    
    
    splitters=['character', 'character', ...]
    non_splitters=['character', 'character', ...]
        
    
    returns ((pre, word, after), (pre, word, after), ...)
    '''
    if splitters is None: splitters = ()
    if non_splitters is None: non_splitters = ()
    w = normalize(norm, words)
    pP = 0
    for i in range(len(w)):
        if category(w[i])[0] not in letter_dia:
            pP += 1
        else:
            break
    preWord = w[0:pP].strip() if pP else ''
    pW = pP
    for i in range(pP, len(w)):
        if w[i] in non_splitters:
            break
        elif category(w[i])[0] in letter_dia:
            pW += 1
        else:
            break
    word = w[pP:pW]
    pA = pW
    nsplit = False
    for i in range(pW, len(w)):
        if clean:
            if category(w[i])[0] in space:
                pA += 1
                break
            elif w[i] in splitters:
                pA += 1
                break
            elif category(w[i])[0] in letter_dia:
                pW = i + 1
                pA = pW
                word += w[i]
            elif category(w[i])[0] not in letter_dia:
                pA += 1
        else:
            if category(w[i])[0] in space:
                pA += 1
                break
            elif w[i] in non_splitters:
                nsplit = True
                continue
            elif category(w[i])[0] not in letter_dia:
                nsplit = False
                pA += 1
            elif category(w[i])[0] in letter_dia and nsplit == True:
                pW = i + 1
                pA = pW
                word += w[i]
            else:
                break
    afterWord = w[pW:pA].strip()
    rest = splitPunc(w[pA:], clean=clean, splitters=splitters, 
                     non_splitters=non_splitters) if pA < len(w) else ()
    return ((preWord, word, afterWord),) + rest

In [ ]:
print(test2)
splitPunc(test2, clean=False, non_splitters=('-', '/',))

In [ ]:
def cleanWords(words, norm=udnorm, clean=False,
               splitters=None, non_splitters=None): 
    """cleanWords splits off any punctuation and 
    non-word characters from words in a string. 
    It can be used for cleaning single words,
    or to tokenize full sentences.
    
    clean=False:
        letter characters that have punctuation
        inbetween but no space, are split on punctuation
        exceptions can be defined in non_splitters
    
    clean=True:
        words with punctuation within (without whitespace) 
        are glued together without punctuation
        exceptions can be defined in splitters
    
    returns: ('string', 'string', ...)
    """
    if splitters is None: splitters = ()
    if non_splitters is None: non_splitters = ()
    w = normalize(norm, words)
    pP = 0
    for i in range(len(w)):
        if category(w[i])[0] not in letter_dia:
            pP += 1
        else:
            break
    pW = pP
    for i in range(pP, len(w)):
        if category(w[i])[0] in letter_dia:
            pW += 1
        else:
            break
    realWord = w[pP:pW]
    pA = pW
    nsplit = False
    for i in range(pW, len(w)):
        if clean:
            if category(w[i])[0] in space:
                break
            elif w[i] in splitters:
                break
            elif category(w[i])[0] not in letter_dia:
                pA += 1
            elif category(w[i])[0] in letter_dia:
                realWord += w[i]
                pA += 1
        else:
            if w[i] in non_splitters:
                nsplit = True
                continue
            elif category(w[i])[0] in letter_dia and nsplit == True:
                pW = i + 1
                pA = pW
                realWord += w[i]
            elif category(w[i])[0] not in letter_dia:
                nsplit = False
                pA += 1
            else:
                break
    res = (realWord,) + \
          (cleanWords(w[pA:], clean=clean, 
                      splitters=splitters, non_splitters=non_splitters) 
           if pA < len(w) else ())
    return res if not res == ('',) else ()


In [ ]:
print(test2)
cleanWords(test2, clean=True,
              splitters=('-'), non_splitters=('-', '/'))

In [ ]:
def tokenizer(sentence, norm=udnorm, punc=False, clean=False,
              splitter=None, non_splitter=None, func=None):
    """tokenize feeds a sentence string
    to splitWord, while concatenating the
    resulting strings into one tuple.
    
    clean=False:
        split on punctuation without whitespace
    clean=True:
        delete punctuation inside words
    clean=None
        
    returns: ('string', 'string', ...)
    """
    if func:
        func(sentence)
    else:
        if punc:
            if clean:
                return tuple(f'{pre}{word}{post}' \
                    for pre, word, post in splitPunc(sentence, norm=udnorm, clean=True,
                                                     splitter=splitter, non_splitter=non_splitter))
            else:
                return tuple(f'{pre}{word}{post}' \
                    for pre, word, post in splitPunc(sentence, norm=udnorm, clean=False,
                                                     splitter=splitter, non_splitter=non_splitter))
        else:
            if clean:
                return cleanWords(sentence, clean=True)
            else:
                return cleanWords(sentence, clean=False)


        

In [1]:
error = {'punc': True, 'nop': False,}

def func(word, **kwargs):
    if kwargs['punc']:
        print(word.lower())

func('DEZE!', **error)

deze!


In [ ]:
dictio = {'publicationStmt': {'concat': True, 'delimit': ', ', 'end': '.'}}
for i in dictio:
    if 'end' in dictio[i]:
        print(i)
        print(dictio[i]['end'])

In [ ]:
import operator

attribs = {'a': 4, 'b': 1, 'c': 25, 'd': 1}
sorted_attribs = sorted(attribs.items(), key=operator.itemgetter(1))
print(sorted_attribs)

In [5]:
from helpertools.unicodetricks import plainLow
from helpertools.data.greek_elisions import ELISIONS

ELISIONS_norm = {k.strip('᾽'): v for k, v in ELISIONS.items()}

def greekReplacements(word):
    if word in ELISION_norm:
        return ELISION_norm[word]
    plain_word = plainLow(word)
    # Deletion of movable-nu
    if plain_word.endswith(('εν', 'σιν', 'στιν')) and len(midWord_pl) >= 3:
        return word[:-1]
    # Handling final-sigma
    if plain_word.endswith('σ'):
        return word[:-1] + 'ς'
    # Handling various forms of ου
    if plain_word in ('ουχ', 'ουκ'):
        return word[:-1]
    # Handling ἐξ
    if plain_word == 'εξ':
        return word[:-1] + 'κ'
    
print(len(ELISIONS))
print(len(ELISIONS_norm))

472
471
